In [1]:
import pandas as pd
import numpy as np
import folium as fl
from geopy.geocoders import Nominatim
import requests
import matplotlib.pyplot as plt
import os
import matplotlib.colors as colors
import matplotlib.cm as cm
from sklearn.cluster import KMeans
import json
from opencage.geocoder import OpenCageGeocode

In [9]:
key='4d00f20c5ff3475cb202c90fa9fd741f'
geocoder=OpenCageGeocode(key)

In [10]:
df_amster=pd.read_csv('/home/aman/Downloads/Amsterdam/Amsterdam_Neighbourhoods.csv')
df_amster.head()

,Neighbourhoods,Latitude,Longitude
0,Admiralenbuurt,52.372734,4.856363
1,Amsteldorp,52.360540,4.905160
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030


In [11]:
df_amster.shape

(105, 3)

In [12]:
df_amster.rename(columns={'Latitude_ams':'Latitude','Longitude_ams':'Longitude'},inplace=True)
df_amster.head()

,Neighbourhoods,Latitude,Longitude
0,Admiralenbuurt,52.372734,4.856363
1,Amsteldorp,52.360540,4.905160
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030


In [13]:
CLIENT_ID='DWUR0ASOGX5MGFQAEJDANEX2C0UQ03DFJQ2PDDHNTX2A0PGV'
CLIENT_SECRET='X0ICJGDGXQEVEKNMVYGNDPUJOTYA3PIABI2POXXSVGHIP0RK'
VERSION='20191105'
LIMIT='30'
print('Your credential:')
print('Client id',CLIENT_ID)
print('Client secret',CLIENT_SECRET)

Your credential:
Client id DWUR0ASOGX5MGFQAEJDANEX2C0UQ03DFJQ2PDDHNTX2A0PGV
Client secret X0ICJGDGXQEVEKNMVYGNDPUJOTYA3PIABI2POXXSVGHIP0RK


In [16]:
address = 'Amsterdam, Netherlands'

geolocator = Nominatim(user_agent="am_explorer")
location = geolocator.geocode(address)
latitude_ams = location.latitude
longitude_ams = location.longitude
print('The geograpical coordinate of Amsterdam are {}, {}.'.format(latitude_ams, longitude_ams))

The geograpical coordinate of Amsterdam are 52.3745403, 4.89797550561798.


In [17]:
map_amsterdam= fl.Map(location=[latitude_ams, longitude_ams],zoom_start=10)#map for the amsterdam using folium
#add marker to the map
for lat, lng,neighborhood in zip(df_amster['Latitude'], df_amster['Longitude'], df_amster['Neighbourhoods']):
    label = '{}, {}'.format(neighborhood,lat,lng)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
    
map_amsterdam

In [10]:
#load the data
with open('/home/aman/Downloads/new_york_data.json') as json_data:
    new_york_data=json.load(json_data)

In [11]:
new_york_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [12]:
neighborhood_data=new_york_data['features']

In [13]:
neighborhood_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [14]:
#transform the data into the pandas dataframe 
column_name=['Borough', 'Neighborhood', 'Latitude', 'Longitude'] #define the column names for the data frame
neighborhoods=pd.DataFrame(columns=column_name)

In [15]:
neighborhoods #this is the empty dataframe which we need to fill

,Borough,Neighborhood,Latitude,Longitude


In [16]:
#lets loop the data and fill the data one row at a time
for data in neighborhood_data:
    borough=data['properties']['borough']
    neighborhood_name=data['properties']['name']
    
    latlon=data['geometry']['coordinates']
    lat=latlon[0]
    lon=latlon[1]
    
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': lat,
                                          'Longitude': lon}, ignore_index=True)

In [45]:
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,-73.847201,40.894705
1,Bronx,Co-op City,-73.829939,40.874294
2,Bronx,Eastchester,-73.827806,40.887556
3,Bronx,Fieldston,-73.905643,40.895437
4,Bronx,Riverdale,-73.912585,40.890834
5,Bronx,Kingsbridge,-73.902818,40.881687
6,Manhattan,Marble Hill,-73.910660,40.876551
7,Bronx,Woodlawn,-73.867315,40.898273
8,Bronx,Norwood,-73.879391,40.877224
9,Bronx,Williamsbridge,-73.857446,40.881039


In [18]:
neighborhoods.to_csv('Newyork_data.csv')

In [18]:
df_newyork=pd.read_csv('/home/aman/Downloads/Newyork_data.csv')

In [19]:
df_newyork.head()

,Unnamed: 0,Borough,Neighborhood,Latitude,Longitude
0,0,Bronx,Wakefield,-73.847201,40.894705
1,1,Bronx,Co-op City,-73.829939,40.874294
2,2,Bronx,Eastchester,-73.827806,40.887556
3,3,Bronx,Fieldston,-73.905643,40.895437
4,4,Bronx,Riverdale,-73.912585,40.890834


In [21]:
df_newyork.drop(['Unnamed: 0'],axis=1,inplace=True)

In [22]:
df_newyork.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,-73.847201,40.894705
1,Bronx,Co-op City,-73.829939,40.874294
2,Bronx,Eastchester,-73.827806,40.887556
3,Bronx,Fieldston,-73.905643,40.895437
4,Bronx,Riverdale,-73.912585,40.890834


In [12]:
df_newyork.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,-73.847201,40.894705
1,Bronx,Co-op City,-73.829939,40.874294
2,Bronx,Eastchester,-73.827806,40.887556
3,Bronx,Fieldston,-73.905643,40.895437
4,Bronx,Riverdale,-73.912585,40.890834


In [23]:
address = 'Newyork, USA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_nw = location.latitude
longitude_nw = location.longitude
print('The geograpical coordinate of Newyork are {}, {}.'.format(latitude_nw, longitude_nw))

The geograpical coordinate of Newyork are 40.83975585, -73.9414480148711.


In [24]:
map_newyork= fl.Map(location=[latitude_nw, longitude_nw],zoom_start=10)#map for the amsterdam using folium
#add marker to the map
for lat, lng,neighborhood in zip(df_newyork['Longitude'], df_newyork['Latitude'], df_newyork['Neighborhood']):
    label = '{}, {}'.format(neighborhood,lat,lng)
    label = fl.Popup(label, parse_html=True)
    fl.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [165]:
df_amster.head()

,Neighbourhoods,Latitude,Longitude
0,Admiralenbuurt,52.372734,4.856363
1,Amsteldorp,52.360540,4.905160
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030


### Merge the newyork and amsterdam data

In [49]:
#df_new=pd.concat([df_newyork,df_amster],sort=False)
#df_new

In [77]:
##defines the neareast venues 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
Top_venues_ams = getNearbyVenues(names=df_amster['Neighbourhoods'],
                                   latitudes=df_amster['Latitude'],
                                   longitudes=df_amster['Longitude']
                                  )

Admiralenbuurt


KeyError: 'groups'

In [29]:
#size of Amsterdam venues
print(Top_venues_ams.shape)
Top_venues_ams

(2319, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Admiralenbuurt,52.372734,4.856363,Radijs,52.371049,4.856756,Bistro
1,Admiralenbuurt,52.372734,4.856363,Rein Cityspa,52.371217,4.855969,Spa
2,Admiralenbuurt,52.372734,4.856363,Sapporo Ramen Sora,52.371294,4.855144,Ramen Restaurant
3,Admiralenbuurt,52.372734,4.856363,Café Cook,52.371208,4.852792,Pub
4,Admiralenbuurt,52.372734,4.856363,Deli-caat,52.371221,4.856056,Deli / Bodega
5,Admiralenbuurt,52.372734,4.856363,T's,52.370527,4.853809,Tea Room
6,Admiralenbuurt,52.372734,4.856363,Kattencafé Kopjes,52.370556,4.855507,Pet Café
7,Admiralenbuurt,52.372734,4.856363,Maz Mez,52.371231,4.857968,Middle Eastern Restaurant
8,Admiralenbuurt,52.372734,4.856363,Broodje Daan,52.373456,4.853299,Deli / Bodega
9,Admiralenbuurt,52.372734,4.856363,STACH food,52.371425,4.860017,Grocery Store


In [ ]:
#size of Amsterdam venues
print(Top_venues_ams.shape)
Top_venues_ams

In [30]:
Top_venues_ams.groupby('Neighborhood').count() #count the values in dataframe in Toronto venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Admiralenbuurt,30,30,30,30,30,30
Amsteldorp,30,30,30,30,30,30
Amsterdam Oud-West,30,30,30,30,30,30
Amsterdam Oud-Zuid,30,30,30,30,30,30
Amsterdam Science Park,4,4,4,4,4,4
Apollobuurt,30,30,30,30,30,30
Betondorp,9,9,9,9,9,9
Bijlmermeer,4,4,4,4,4,4
Binnenstad,30,30,30,30,30,30


In [73]:
print('There are {} uniques categories.'.format(len(Top_venues_ams['Venue Category'].unique())))

There are 250 uniques categories.


In [74]:
Top_venues_ny = getNearbyVenues(names=df_newyork1['Neighborhood'],
                                   latitudes=df_newyork1['Latitude'],
                                   longitudes=df_newyork1['Longitude']
                                  )

Wakefield


KeyError: 'groups'

## Analyzing each neighbourhood

In [32]:
# one hot encoding
Top_onehot_ams = pd.get_dummies(Top_venues_ams[['Venue Category']])

# add neighborhood column back to dataframe
Top_onehot_ams['Neighborhood'] = Top_venues_ams['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Top_onehot_ams.columns[-1]] + list(Top_onehot_ams.columns[:-1])
Toronto_onehot_ams = Top_onehot_ams[fixed_columns]

Top_onehot_ams

,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_African Restaurant,Venue Category_Arcade,Venue Category_Argentinian Restaurant,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Entertainment,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,...,Venue Category_Vietnamese Restaurant,Venue Category_Whisky Bar,Venue Category_Windmill,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Women's Store,Venue Category_Yoga Studio,Venue Category_Zoo,Venue Category_Zoo Exhibit,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Admiralenbuurt


## now lets group rows by neighbourhood

In [33]:
Top_grouped_ams = Top_onehot_ams.groupby('Neighborhood').mean().reset_index()
Top_grouped_ams

,Neighborhood,Venue Category_Accessories Store,Venue Category_Afghan Restaurant,Venue Category_African Restaurant,Venue Category_Arcade,Venue Category_Argentinian Restaurant,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Entertainment,Venue Category_Asian Restaurant,...,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Vietnamese Restaurant,Venue Category_Whisky Bar,Venue Category_Windmill,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Women's Store,Venue Category_Yoga Studio,Venue Category_Zoo,Venue Category_Zoo Exhibit
0,Admiralenbuurt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,Amsteldorp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.0
2,Amsterdam Oud-West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,0.0
3,Amsterdam Oud-Zuid,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,Amsterdam Science Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,Apollobuurt,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,Betondorp,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,Bijlmermeer,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
8,Binnenstad,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,Bos en Lommer,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


## Lets print each of the neighbourhood with most common venues

In [34]:
num_top_venues = 5

for hood in Top_grouped_ams['Neighborhood']:
    print("----"+hood+"----")
    temp = Top_grouped_ams[Top_grouped_ams['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Admiralenbuurt----
                                      venue  freq
0                       Venue Category_Café  0.07
1              Venue Category_Deli / Bodega  0.07
2  Venue Category_Middle Eastern Restaurant  0.07
3                        Venue Category_Bar  0.07
4                 Venue Category_Restaurant  0.07


----Amsteldorp----
                              venue  freq
0              Venue Category_Hotel  0.13
1        Venue Category_Coffee Shop  0.10
2  Venue Category_French Restaurant  0.07
3         Venue Category_Restaurant  0.07
4             Venue Category_Bakery  0.07


----Amsterdam Oud-West----
                             venue  freq
0       Venue Category_Coffee Shop  0.10
1              Venue Category_Café  0.10
2     Venue Category_Movie Theater  0.07
3        Venue Category_Restaurant  0.07
4  Venue Category_Dutch Restaurant  0.07


----Amsterdam Oud-Zuid----
                              venue  freq
0              Venue Category_Hotel  0.13
1             Ve

                           venue  freq
0           Venue Category_Hotel  0.13
1      Venue Category_Restaurant  0.07
2             Venue Category_Bar  0.07
3          Venue Category_Museum  0.07
4  Venue Category_Breakfast Spot  0.07


----Haarlemmerbuurt----
                                          venue  freq
0                    Venue Category_Coffee Shop  0.13
1  Venue Category_Vegetarian / Vegan Restaurant  0.07
2                            Venue Category_Bar  0.07
3                         Venue Category_Bakery  0.07
4             Venue Category_Italian Restaurant  0.03


----Hoofddorppleinbuurt----
                               venue  freq
0                Venue Category_Café  0.10
1         Venue Category_Coffee Shop  0.07
2  Venue Category_Italian Restaurant  0.07
3          Venue Category_Restaurant  0.07
4      Venue Category_Breakfast Spot  0.07


----Houthaven----
                        venue  freq
0      Venue Category_Theater  0.25
1   Venue Category_Board Shop  0.12


                         venue  freq
0   Venue Category_Supermarket  0.33
1          Venue Category_Pier  0.17
2   Venue Category_Flower Shop  0.17
3           Venue Category_Gym  0.17
4  Venue Category_Soccer Field  0.17


----Osdorp----
                                  venue  freq
0         Venue Category_Ice Cream Shop  0.11
1           Venue Category_Neighborhood  0.11
2  Venue Category_Indonesian Restaurant  0.11
3          Venue Category_Grocery Store  0.11
4           Venue Category_Dessert Shop  0.11


----Oud Osdorp----
                                  venue  freq
0         Venue Category_Ice Cream Shop  0.11
1           Venue Category_Neighborhood  0.11
2  Venue Category_Indonesian Restaurant  0.11
3          Venue Category_Grocery Store  0.11
4           Venue Category_Dessert Shop  0.11


----Oud-Oost----
                             venue  freq
0              Venue Category_Café  0.10
1       Venue Category_Coffee Shop  0.07
2            Venue Category_Bakery  0.07
3    

                                       venue  freq
0  Venue Category_Construction & Landscaping   0.5
1               Venue Category_Boat or Ferry   0.5
2           Venue Category_Accessories Store   0.0
3                        Venue Category_Park   0.0
4                   Venue Category_Nightclub   0.0


----Westelijke Eilanden----
                                      venue  freq
0                 Venue Category_Restaurant  0.13
1              Venue Category_Deli / Bodega  0.07
2                Venue Category_Music Venue  0.03
3                 Venue Category_Beer Store  0.03
4  Venue Category_Middle Eastern Restaurant  0.03


----Westelijke Tuinsteden----
                           venue  freq
0           Venue Category_Hotel  0.13
1      Venue Category_Restaurant  0.07
2             Venue Category_Bar  0.07
3          Venue Category_Museum  0.07
4  Venue Category_Breakfast Spot  0.07


----Westerpark----
                               venue  freq
0  Venue Category_Italian Restaura

### Convert the data into dataframe

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Top 10 venues for each neighbourhood

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_ams = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_ams['Neighborhood'] = Top_grouped_ams['Neighborhood']

for ind in np.arange(Top_grouped_ams.shape[0]):
    neighborhoods_venues_sorted_ams.iloc[ind, 1:] = return_most_common_venues(Top_grouped_ams.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_ams

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiralenbuurt,Venue Category_Middle Eastern Restaurant,Venue Category_Bar,Venue Category_Deli / Bodega,Venue Category_Restaurant,Venue Category_Café,Venue Category_Scandinavian Restaurant,Venue Category_French Restaurant,Venue Category_Bistro,Venue Category_Fish Market,Venue Category_Spa
1,Amsteldorp,Venue Category_Hotel,Venue Category_Coffee Shop,Venue Category_Steakhouse,Venue Category_French Restaurant,Venue Category_Restaurant,Venue Category_Bakery,Venue Category_Italian Restaurant,Venue Category_Marijuana Dispensary,Venue Category_Dessert Shop,Venue Category_Theater
2,Amsterdam Oud-West,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Movie Theater,Venue Category_Dutch Restaurant,Venue Category_Restaurant,Venue Category_Italian Restaurant,Venue Category_Pizza Place,Venue Category_Brewery,Venue Category_Breakfast Spot,Venue Category_Bookstore
3,Amsterdam Oud-Zuid,Venue Category_Hotel,Venue Category_Restaurant,Venue Category_Bakery,Venue Category_French Restaurant,Venue Category_Comedy Club,Venue Category_Gastropub,Venue Category_Plaza,Venue Category_Concert Hall,Venue Category_Deli / Bodega,Venue Category_Chinese Restaurant
4,Amsterdam Science Park,Venue Category_Coffee Shop,Venue Category_Convenience Store,Venue Category_Restaurant,Venue Category_Doner Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Field,Venue Category_Fast Food Restaurant,Venue Category_Farmers Market,Venue Category_Farm,Venue Category_Falafel Restaurant
5,Apollobuurt,Venue Category_Hotel,Venue Category_Italian Restaurant,Venue Category_Art Gallery,Venue Category_Bakery,Venue Category_Bistro,Venue Category_Indonesian Restaurant,Venue Category_Tapas Restaurant,Venue Category_Theater,Venue Category_Bar,Venue Category_Bagel Shop
6,Betondorp,Venue Category_Furniture / Home Store,Venue Category_Market,Venue Category_Seafood Restaurant,Venue Category_Museum,Venue Category_Dutch Restaurant,Venue Category_Historic Site,Venue Category_Art Gallery,Venue Category_Event Space,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant
7,Bijlmermeer,Venue Category_Park,Venue Category_Playground,Venue Category_Harbor / Marina,Venue Category_Nightclub,Venue Category_Zoo Exhibit,Venue Category_Fabric Shop,Venue Category_Dutch Restaurant,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant,Venue Category_Event Space
8,Binnenstad,Venue Category_Hotel,Venue Category_Bar,Venue Category_Cocktail Bar,Venue Category_History Museum,Venue Category_Soup Place,Venue Category_Mexican Restaurant,Venue Category_Café,Venue Category_Marijuana Dispensary,Venue Category_Chinese Restaurant,Venue Category_Sushi Restaurant
9,Bos en Lommer,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Bar,Venue Category_Bakery,Venue Category_Coffee Shop,Venue Category_Indonesian Restaurant,Venue Category_Performing Arts Venue,Venue Category_Bistro,Venue Category_Sandwich Place,Venue Category_Supermarket,Venue Category_Bookstore


## Applying K-Means clustering for the venues

In [37]:
# set number of clusters
kclusters = 8

Top_grouped_ams_clustering = Top_grouped_ams.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Top_grouped_ams_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:37] 

array([3, 1, 3, 1, 5, 1, 3, 0, 1, 3, 6, 0, 0, 3, 7, 1, 3, 1, 3, 3, 1, 3,
       1, 3, 3, 3, 0, 1, 3, 0, 1, 0, 1, 3, 3, 0, 0])

### Lets create a new dataframe that contains cluster as well as top 10 venues for each neighourhood

In [40]:
# add clustering labels
#neighborhoods_venues_sorted_ams.insert(0, 'Cluster Labels', kmeans.labels_)

Top_merged_ams = df_amster

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Top_merged_ams = Top_merged_ams.join(neighborhoods_venues_sorted_ams.set_index('Neighborhood'), on='Neighbourhoods')

Top_merged_ams # check the last columns!

,Neighbourhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiralenbuurt,52.372734,4.856363,3,Venue Category_Middle Eastern Restaurant,Venue Category_Bar,Venue Category_Deli / Bodega,Venue Category_Restaurant,Venue Category_Café,Venue Category_Scandinavian Restaurant,Venue Category_French Restaurant,Venue Category_Bistro,Venue Category_Fish Market,Venue Category_Spa
1,Amsteldorp,52.360540,4.905160,1,Venue Category_Hotel,Venue Category_Coffee Shop,Venue Category_Steakhouse,Venue Category_French Restaurant,Venue Category_Restaurant,Venue Category_Bakery,Venue Category_Italian Restaurant,Venue Category_Marijuana Dispensary,Venue Category_Dessert Shop,Venue Category_Theater
2,Amsterdam Oud-West,52.365390,4.870220,3,Venue Category_Café,Venue Category_Coffee Shop,Venue Category_Movie Theater,Venue Category_Dutch Restaurant,Venue Category_Restaurant,Venue Category_Italian Restaurant,Venue Category_Pizza Place,Venue Category_Brewery,Venue Category_Breakfast Spot,Venue Category_Bookstore
3,Amsterdam Oud-Zuid,52.352350,4.877880,1,Venue Category_Hotel,Venue Category_Restaurant,Venue Category_Bakery,Venue Category_French Restaurant,Venue Category_Comedy Club,Venue Category_Gastropub,Venue Category_Plaza,Venue Category_Concert Hall,Venue Category_Deli / Bodega,Venue Category_Chinese Restaurant
4,Amsterdam Science Park,52.354320,4.958030,5,Venue Category_Coffee Shop,Venue Category_Convenience Store,Venue Category_Restaurant,Venue Category_Doner Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Field,Venue Category_Fast Food Restaurant,Venue Category_Farmers Market,Venue Category_Farm,Venue Category_Falafel Restaurant
5,Apollobuurt,52.350298,4.868131,1,Venue Category_Hotel,Venue Category_Italian Restaurant,Venue Category_Art Gallery,Venue Category_Bakery,Venue Category_Bistro,Venue Category_Indonesian Restaurant,Venue Category_Tapas Restaurant,Venue Category_Theater,Venue Category_Bar,Venue Category_Bagel Shop
6,Betondorp,52.423407,4.833396,3,Venue Category_Furniture / Home Store,Venue Category_Market,Venue Category_Seafood Restaurant,Venue Category_Museum,Venue Category_Dutch Restaurant,Venue Category_Historic Site,Venue Category_Art Gallery,Venue Category_Event Space,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant
7,Bijlmermeer,52.309891,4.974420,0,Venue Category_Park,Venue Category_Playground,Venue Category_Harbor / Marina,Venue Category_Nightclub,Venue Category_Zoo Exhibit,Venue Category_Fabric Shop,Venue Category_Dutch Restaurant,Venue Category_Electronics Store,Venue Category_Ethiopian Restaurant,Venue Category_Event Space
8,Binnenstad,52.372160,4.904370,1,Venue Category_Hotel,Venue Category_Bar,Venue Category_Cocktail Bar,Venue Category_History Museum,Venue Category_Soup Place,Venue Category_Mexican Restaurant,Venue Category_Café,Venue Category_Marijuana Dispensary,Venue Category_Chinese Restaurant,Venue Category_Sushi Restaurant
9,Bos en Lommer,52.379190,4.851740,3,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Bar,Venue Category_Bakery,Venue Category_Coffee Shop,Venue Category_Indonesian Restaurant,Venue Category_Performing Arts Venue,Venue Category_Bistro,Venue Category_Sandwich Place,Venue Category_Supermarket,Venue Category_Bookstore


In [154]:
#Top_merged['Cluster Labels'].fillna(0, inplace=True)

In [155]:
#Top_merged["Cluster Labels"]= Top_merged["Cluster Labels"].astype(int)

In [42]:
# create map
map_clusters_ams = fl.Map(location=(latitude_ams,longitude_ams),zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Top_merged_ams['Latitude'], Top_merged_ams['Longitude'], Top_merged_ams['Neighbourhoods'], Top_merged_ams['Cluster Labels']):
    label = fl.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    fl.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_ams)
       
map_clusters_ams